# Monopoly Game Agent
Proof of Concept

In [ ]:
%pip install -U langchain-community langchain-chroma langchain-openai

In [57]:
from langchain_openai import ChatOpenAI

# Create a new model
model = ChatOpenAI(model="gpt-4o")

### Game Functions
Wrappers that retrieves relevant game state(s)

In [84]:
from simulator.monosim.player import Player
from simulator.monosim.board import get_board, get_roads, get_properties, get_community_chest_cards, get_bank

In [85]:
"""
Initialize game with 2 players and return the general board info 
(bank, board, roads, properties, community_chest_cards)
"""
def initialize_game():
    bank = get_bank()
    board = get_board()
    roads = get_roads()
    properties = get_properties()
    community_chest_cards = get_community_chest_cards()
    community_cards_deck = list(community_chest_cards.keys())

    player1 = Player('player1', 1, bank, board, roads, properties, community_cards_deck)
    player2 = Player('player2', 2, bank, board, roads, properties, community_cards_deck)
    
    player1.meet_other_players([player2])
    player2.meet_other_players([player1])
    
    return {
        "bank": bank,
        "board": board,
        "roads": roads,
        "properties": properties,
        "community_chest_cards": community_chest_cards,
        "players": [player1, player2] # For now, player 1 always comes first
    }

In [86]:
initial_state = initialize_game()
initial_state["bank"]

{'cash': 5000, 'houses': 32, 'hotels': 12}

In [129]:
"""
Get the player's state, including position, roads owned, money, mortgaged properties, etc.
"""
def get_current_state(players):
    current_state = {
        "players": [{"state": player.get_state()} for player in players]
    }
    return current_state

### Prompt Template

In [88]:
agent_role = "Player 1" # we can switch the role of the agent to Player 2

In [142]:
from langchain import OpenAI, LLMChain, PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain_core.runnables import RunnableWithMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import BaseMessage, AIMessage
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory
)

from typing import List



In [143]:
# NOTE：For the sake of POC, currently it only supports:
# - 2 players
# - buy property or not: yes / no
initial_template = """
  You are the {agent_role} in a Monopoly game. Here is the current game state:

  Bank:
  {initial_bank}

  Board:
  {initial_board}

  Roads:
  {initial_roads}

  Properties:
  {initial_properties}

  Players:
  Player 1 and Player 2

  Your Objective:
  Given the current state of the game, make strategic moves that maximizes your chances of winning.

  Guidelines:
  1. Analyze each component of the game state to understand your current situation.
  2. Consider any immediate risks or opportunities from property ownership, player positions, or your current balance.

  Instructions:
  - Reason step-by-step to ensure your action aligns with the game’s rules and overall strategy.
  - Provide your next move by determining if you should buy the property or not.
"""

initial_prompt = PromptTemplate(
    input_variables=["agent_role", "initial_bank", "initial_board", "initial_roads", "initial_properties"],
    template=initial_template
)


In [144]:
# Dynamic prompt template to update the player state only (we can also only show the previous moves)
dynamic_template = """
  Current Player State:
  {player_state}

  Based on the initial setup and current state, what is your next move?
"""

dynamic_prompt = PromptTemplate(
    input_variables=["player_state"],
    template=dynamic_template
)

def get_dynamic_prompt(player_state):
    return dynamic_template.format(player_state=player_state)

### Structure Output
Experimenting with Langchain's `pydantic_v1` to ensure more structured data

In [145]:
from pydantic import BaseModel, Field

In [146]:
class Output(BaseModel):
    decision: str = Field(description="Your decision for the next move")
    reasoning: str = Field(description="Your reasoning for the decision")

### Memory

In [147]:
class InMemoryHistory(BaseChatMessageHistory, BaseModel):
    """In memory implementation of chat message history."""

    messages: List[BaseMessage] = Field(default_factory=list)

    def add_messages(self, messages: List[BaseMessage]) -> None:
        """Add a list of messages to the store"""
        self.messages.extend(messages)

    def clear(self) -> None:
        self.messages = []

In [148]:
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

### Simulate the Game

Starting the game

In [149]:
game = initialize_game()

In [150]:
# Initialize memory to retain the initial static prompt context
memory = ConversationBufferMemory()

# Inject the static context once into memory
initial_context = initial_template.format(
    agent_role="Player 1",  # or as appropriate
    initial_bank=game["bank"],
    initial_board=game["board"],
    initial_roads=game["roads"],
    initial_properties=game["properties"]
)

memory.save_context({"input": ""}, {"output": initial_context})

In [151]:
structured_llm = model.with_structured_output(Output)
llm_with_memory = RunnableWithMessageHistory(
    structured_llm,
    get_session_history,
)

chain = (dynamic_prompt | llm_with_memory)

In [152]:
player1 = game["players"][0]
player2 = game["players"][1]
list_players = [player1, player2]

stop_at_round = 5 # arbitrary number of rounds to play before agent comes in and make a decision (for POC)

idx_count = 0
while not player1.has_lost() and not player2.has_lost() and idx_count < stop_at_round:
    for player in list_players:
        player.play()
    idx_count += 1


### Get the Result
Note: we can't guarantee that the AI is on a property after some moves, hence I omitted the last part of the logic for triggering `buy_property()`

In [157]:
import json

config = {"configurable": {"session_id": "123"}}
player_state = get_current_state(list_players)

player_state_str = json.dumps(player_state, indent=2)

response = chain.invoke(
    "player_state_str",
    config={"configurable": {"session_id": "123"}}
)

ValueError: Expected str, BaseMessage, List[BaseMessage], or Tuple[BaseMessage]. Got text='\n  Current Player State:\n  bruh\n\n  Based on the initial setup and current state, what is your next move?\n'.